In [1]:
from tunable_agents import utility, agent
import utils
import gin
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tunable_agents.environments.gathering_env import gathering_env
import matplotlib.pyplot as plt
from scipy.stats import wasserstein_distance

%matplotlib inline

In [ ]:
path = "results/reward_vector/"
for file in os.listdir(path):
    if os.path.isfile(path + file):
        shape = np.load(path + file, allow_pickle=True).shape
        print(f"{file:<60}     {shape}")

# Scalar Utilities

In [2]:
path = "results/utilities/"

results = utils.load_results(path)
keys = sorted(results.keys())

In [3]:
intra_wds = dict()
for key in keys:
    intra_wds[key] = [wasserstein_distance(results[key][i], results[key][j])
                      for i in range(len(results[key])) for j in range(i+1, len(results[key]))]

inter_wds = dict()
for i in range(len(keys)):
    for j in range(i+1, len(keys)):
        key_1, key_2 = keys[i], keys[j]
        if key_1.split("-")[0] == key_2.split("-")[0]:
            inter_wds[key_1 + "--" + key_2] = [wasserstein_distance(result_1, result_2)
                                            for result_1 in results[key_1] for result_2 in results[key_2]]

for key in keys:
    mean_WD = np.round(np.mean(intra_wds[key]), 4)
    print(f"{key:<40}     {mean_WD}")

cum_rewards_env-128_128_64_model             0.987
cum_rewards_env-64_64_model                  1.327
cum_threshold_env-128_128_64_model           1.4577
cum_threshold_env-64_64_model                1.9679
replication_env-128_128_64_model             0.7244
replication_env-64_64_model                  1.2287
threshold_env-128_128_64_model               2.0162
threshold_env-64_64_model                    4.303


In [4]:
for key in inter_wds.keys():
    mean_WD = np.round(np.mean(inter_wds[key]), 4)
    print(f"{key:<70}     {mean_WD}")

cum_rewards_env-128_128_64_model--cum_rewards_env-64_64_model              3.1498
cum_threshold_env-128_128_64_model--cum_threshold_env-64_64_model          1.6634
replication_env-128_128_64_model--replication_env-64_64_model              2.9361
threshold_env-128_128_64_model--threshold_env-64_64_model                  5.0472


In [5]:
print("{:^30}            {:^12}      {:^10}".format("Model", "Mean", "Standard Error"))
for key in keys:
    res = [np.mean(result) for result in results[key]]
    mean = np.round(np.mean(res), 4)
    std_err = np.round(np.std(res)/np.sqrt(len(res)), 4)
    print(f"{key:<40}     {str(mean):<12}       {str(std_err):<30}")

            Model                             Mean          Standard Error
cum_rewards_env-128_128_64_model             14.9522            0.2754                        
cum_rewards_env-64_64_model                  11.8223            0.4006                        
cum_threshold_env-128_128_64_model           8.4736             0.4169                        
cum_threshold_env-64_64_model                7.5494             0.6252                        
replication_env-128_128_64_model             14.3529            0.1786                        
replication_env-64_64_model                  11.436             0.3531                        
threshold_env-128_128_64_model               8.2447             0.642                         
threshold_env-64_64_model                    3.4226             1.3389                        


In [6]:
for key in keys:
    result_list = [np.mean(result) for result in results[key]]
    print(f"{key:<40} {result_list}")

cum_rewards_env-128_128_64_model         [13.910725, 15.17225, 15.2183, 14.432275, 14.9077, 16.07195]
cum_rewards_env-64_64_model              [11.6996, 12.63365, 11.8517, 10.0577, 13.19475, 11.496625]
cum_threshold_env-128_128_64_model       [8.889875, 7.914425, 7.134875, 9.937125, 9.4408, 7.524575]
cum_threshold_env-64_64_model            [8.764525, 7.88765, 8.67945, 8.734775, 6.6912, 4.539075]
replication_env-128_128_64_model         [13.912125, 14.477425, 13.91235, 14.568975, 15.14865, 14.0978]
replication_env-64_64_model              [12.474725, 10.209075, 10.3794, 11.6756, 11.603375, 12.273975]
threshold_env-128_128_64_model           [5.01925, 7.8694, 9.491775, 9.701975, 9.0186, 8.367375]
threshold_env-64_64_model                [-2.852275, 4.206975, 1.9436, 6.92235, 3.726675, 6.588025]


# Fixed Environment Evaluation

In [6]:
path = "results/fixed_env_results/"

results = utils.load_results(path)
keys = sorted(results.keys())

In [7]:
for key in keys:
    uniques = [np.unique(result, axis=0).shape[0] for result in results[key]]
    mean_uniques = np.round(np.mean(uniques), 1)
    print(f"{key:<60} {str(uniques):<40} {mean_uniques}")

cum_rewards_env-128_128_64_model                             [183, 129, 99, 150, 148, 128]            139.5
cum_rewards_env-256_128_128_64_64_model                      [128, 158, 169, 130, 159, 176]           153.3
cum_rewards_env-64_64_model                                  [158, 106, 133, 140, 67, 139]            123.8
cum_rewards_env-more_utils_128_128_64_model                  [293, 192, 154, 267, 231, 196]           222.2
cum_rewards_env-more_utils_256_128_128_64_64_model           [235, 268, 246, 190, 308, 304]           258.5
cum_rewards_env-more_utils_64_64_model                       [246, 174, 195, 219, 111, 211]           192.7
cum_threshold_env-128_128_64_model                           [306, 205, 286, 236, 192, 183]           234.7
cum_threshold_env-64_64_model                                [154, 216, 277, 111, 160, 247]           194.2
replication_env-128_128_64_model                             [111, 116, 59, 221, 106, 169]            130.3
replication_env-256_128_128_

# Stale Analysis

In [ ]:
path = "results/legacy_results/"

results_files = {"david": ["david_200221.npy", "david_210221.npy", "david_210221_2.npy"],
                 "my_env": ["my_210220.npy", "my_210221.npy", "my_210221_2.npy"],
                 "new": ["new_gath_repl_1.npy", "new_gath_repl_2.npy", "new_gath_repl_3.npy"],
                 "tfa": ["gathering_replication_1.npy", "gathering_replication_2.npy", "gathering_replication_3.npy",
                         "gathering_replication_4.npy"]


keys = sorted(results_files.keys())
results = dict()
for key in keys:
    results[key] = [np.load(path + file, allow_pickle=True) for file in results_files[key]]

if True:
    for key in keys:
        results[key] = [result[result != -1] for result in results[key]]


In [ ]:
'''
plt.figure(figsize=(15, 12))
i = 1
for result in results["my_env"] + results["david"] + results["tfa"][:-1]:
    plt.subplot(3, 3, i)
    plt.title(f"Number of sampled episodes: {result.shape[0]}")
    plt.hist(result, bins=200, density=True)
    i += 1

plt.tight_layout()
plt.show()
'''
zz =0